#### Cine21 영화배우 정보 크롤링
* site 주소: http://www.cine21.com/rank/person/
* 개발자 도구에서 xhr인 것만 필터링 network -> content 
* Reqeust URL : http://www.cine21.com/rank/person/content
* Reqeust method: post
* form data
- section: actor
- period_Start:2020-07 
- gender: all
- page:1
* 1~2년치 데이터를 한꺼번에 크롤링 해서 actors.json 파일로 저장하기

In [13]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin

### 정규표현식
* \w는 문자와 숫자
* *: 0~ n (횟수), +:1~n, ?:0(zero), 

* .(period) : 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다.
* .* 는 문자가 0번 또는 그 이상 반복되는 패턴
* <.*> 태그안에 있는 .*
* Greedy(.*): 모든 태그선택 vs Non-Greedy (.*?): 첫번째 매칭되는 태그만 선택
* [\n\r\t] : \n,\r,\t  중 하나는 찾는다
* re.compile(r'[\n\r\t]') 는 해당 정규표현식을 찾아주는 컴파일러 생성해주는 함수
* 컴파일 함수 내의 정규표현식 앞에는 항상 r 붙인다

In [ ]:
<pre>
*: 0~ n (횟수), +:1~n, ?:0(zero)
 </pre>

### 1단계

In [59]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict=dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'
formdata_dict['page'] = 1
formdata_dict

#post
res = requests.post(actor_url, data=formdata_dict)
print(res.ok)
soup = BeautifulSoup(res.text, 'html.parser')

actor_item_list = list()

#css선택자 이용해서 데이터 뽑아내기
for actor in soup.select('li.people_li div.name' ):
    #print(actor)
    #print(re.sub('\(\w*\)', '', actor.text))      #(편) : 괄호 빼주기
    
    
    
    for a_tag in soup.select('li.people_li div.name a'):
        #print(a_tag['href'])
        actor_detail_url = urljoin(actor_url,a_tag['href'] )
        #print(actor_detail_url) # 상세 페이지 나옴

        res = requests.get(actor_detail_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup

        actor_item_dict = dict()
        for li_tag in soup.select('ul.default_info li'): 
            
            #print(li_tag) 
            #print(li_tag.text)
            ## dict의 key에 해당하는 값을 추출한다. dict['직업']
            ##<li><span class="tit">직업</span>배우</li>
            actor_item_field = li_tag.select_one('span.tit').text # w직업 생년월일 성별 등 ..
            
            #print(actor_item_field)    
            # 이젠 직업은 추출했으니 배우를 추출해보자
            # <span.*?>.*?</span>를 ''으로 sub 하자!!
            ## dict의 value 에 해당하는 값 추출. dict['직업']='배우'
            ##<li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) ## str로 싸서 줌 <li>배우</li>
            actor_item_value = re.sub('<.*?>','', actor_item_value) ##배우
            #print(actor_item_value)
            # url 사이의 공백 없애주기 - 멜론에서도 했었ㄷ음?
            
            """ 적용전
               \n http://www.gangdongwon.com
            """
            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('',actor_item_value)
             """ 적용후
               \nhttp://www.gangdongwon.com
            """
            
            # 이 데이터를 리스트에 저장하면 디비에 저장하기 쉽겠지?
            actor_item_dict[actor_item_field] = actor_item_value
            
            #print(actor_item_value)
            #print(actor_item_dict)# 이렇게 하면 누적이 됨
        actor_item_list.append(actor_item_dict)    
      
actor_item_list          

True


[{'직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'직업': '배우', '생년월일': '1982-12-14', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1979-03-20',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/cjesminjae',
  '학교': '한국예술종합학교'},
 {'다른 이름': '엄홍식',
  '직업': '배우',
  '생년월일': '1986-10-06',
  '성별': '남',
  '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik',
  '신장/체중': '178cm

In [21]:
# \w는 문자와 숫자
# *: 0~ n (횟수), +:1~n, ?:0(zero), 
# 정규표현식 사용해서 
test_data='유아인(2편)'
# 정규표현식의 이런 함수, \괄호
re.sub('\(\w*\)','',test_data) #1식을 2로 대체시켜라

'유아인'

In [53]:
temp_data ="""
\n
https://www.gangdongwon.com
\n
학교"""
regex = re.compile(r'[\n\r\t]')
regex.sub('',temp_data)

'https://www.gangdongwon.com학교'

### 2단계

In [76]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict=dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'
formdata_dict['page'] = 1
formdata_dict

#post
res = requests.post(actor_url, data=formdata_dict)
print(res.ok)
soup = BeautifulSoup(res.text, 'html.parser')

actor_item_list = list()
actors =  soup.select('li.people_li div.name' )
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
rankings = soup.select('li.people_li span.grade')

for index, actor in enumerate(actors): #배우랑 흥행지수, 인덱스 =흥행지수, 배우 같음
    actor_item_dict = dict()
        
    
    # 유아인(2편) 에서 (2편)을 제거하고 저장.
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_item_dict['배우이름']=actor_name
    # 흥행지수 값에서 ,를 제거하고 정수타입으로 변환해서 저장, replace() ,를 공백으로
    actor_hit = int(hits[index].text.replace(',',''))
    actor_item_dict['흥행지수'] = actor_hit
    
    
    #출연작, 
    movie_titles = movies[index].select('li a span')
    # 출연작 여러개의 title을 저장하는 리스트- 권해효 3개..
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화']= movie_title_list
    # 순위 
    actor_ranking = rankings[index].text
    actor_item_dict['랭킹']=int(actor_ranking)
    
    #배우의 상세정보위해 http get 다시 요청함
    #print(actor_name, actor_hit)
    #print(actor_name, movie_titles)
    #print(actor_name, movie_title_list)
    #print(actor_name, actor_ranking)
    
#css선택자 이용해서 데이터 뽑아내기
#for actor in soup.select('li.people_li div.name' ):
     
       
        
    #print(actor)
    #print(re.sub('\(\w*\)', '', actor.text))    
    
    
    #for a_tag in soup.select('li.people_li div.name a'):
#         """
#             <div class=name>   <= actor
#                 <a href="/db/person/info/?person_id=18040"> actor_detail_url
#         """
    actor_detail_url = actor.select_one('a').attrs['href']
    actor_detail_full_url = urljoin(actor_url,actor_detail_url )
    #print(actor_detail_url) # 상세 페이지 나옴
    print(actor_detail_full_url)


    res = requests.get(actor_detail_full_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    soup

    for li_tag in soup.select('ul.default_info li'): 

        #print(li_tag) 
        #print(li_tag.text)
        ## dict의 key에 해당하는 값을 추출한다. dict['직업']
        ##<li><span class="tit">직업</span>배우</li>
        actor_item_field = li_tag.select_one('span.tit').text # w직업 생년월일 성별 등 ..

        #print(actor_item_field)    
        # 이젠 직업은 추출했으니 배우를 추출해보자
        # <span.*?>.*?</span>를 ''으로 sub 하자!!
        ## dict의 value 에 해당하는 값 추출. dict['직업']='배우'
        ##<li><span class="tit">직업</span>배우</li>
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) ## str로 싸서 줌 <li>배우</li>
        actor_item_value = re.sub('<.*?>','', actor_item_value) ##배우
        #print(actor_item_value)
        # url 사이의 공백 없애주기 - 멜론에서도 했었ㄷ음?

        """ 적용전
           \n http://www.gangdongwon.com
        """
        regex = re.compile(r'[\n\r\t]')
        actor_item_value = regex.sub('',actor_item_value)
        """ 적용후
           \nhttp://www.gangdongwon.com
        """

        # 이 데이터를 리스트에 저장하면 디비에 저장하기 쉽겠지?
        actor_item_dict[actor_item_field] = actor_item_value

        #print(actor_item_value)
        #print(actor_item_dict)# 이렇게 하면 누적이 됨
    actor_item_list.append(actor_item_dict)    
      
#actor_item_list        

True
http://www.cine21.com/db/person/info/?person_id=18040
http://www.cine21.com/db/person/info/?person_id=2760
http://www.cine21.com/db/person/info/?person_id=87306
http://www.cine21.com/db/person/info/?person_id=167
http://www.cine21.com/db/person/info/?person_id=71787
http://www.cine21.com/db/person/info/?person_id=40151
http://www.cine21.com/db/person/info/?person_id=57657


[{'배우이름': '강동원',
  '흥행지수': 159665,
  '출연영화': ['반도'],
  '랭킹': 1,
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'배우이름': '이정현',
  '흥행지수': 139707,
  '출연영화': ['반도'],
  '랭킹': 2,
  '직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'배우이름': '이레',
  '흥행지수': 119748,
  '출연영화': ['반도'],
  '랭킹': 3,
  '직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'배우이름': '권해효',
  '흥행지수': 99795,
  '출연영화': ['반도', '사이비', '한낮의 피크닉'],
  '랭킹': 4,
  '직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'배우이름': '구교환',
  '흥행지수': 79832,
  '출연영화': ['반도'],
  '랭킹':

### 3단계
* 2단계 : 1page의 7명의 배우의 정보를 출력
* 1, 2, 3 페이지의 21명의 배우의 정보를 출력

In [81]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict=dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'


actor_item_list = list()


for page in range(1, 4):
    formdata_dict['page'] = page
    print('--->{}페이지 출력됨'.format(page))
    #post
    res = requests.post(actor_url, data=formdata_dict)
    #print(res.ok)
    soup = BeautifulSoup(res.text, 'html.parser')

    
    actors =  soup.select('li.people_li div.name' )
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li span.grade')

    for index, actor in enumerate(actors): #배우랑 흥행지수
        print('===>{}번째 배우 출력됨'.format(index))
        actor_item_dict = dict()


        # 유아인(2편) 에서 (2편)을 제거하고 저장.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dict['배우이름']=actor_name
        # 흥행지수 값에서 ,를 제거하고 정수타입으로 변환해서 저장, replace() ,를 공백으로
        actor_hit = int(hits[index].text.replace(',',''))
        actor_item_dict['흥행지수'] = actor_hit


        #출연작, 
        movie_titles = movies[index].select('li a span')
        # 출연작 여러개의 title을 저장하는 리스트- 권해효 3개..
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
            actor_item_dict['출연영화']= movie_title_list
        # 순위 
        actor_ranking = rankings[index].text
        actor_item_dict['랭킹']=int(actor_ranking)


    #         """
    #             <div class=name>   <= actor
    #                 <a href="/db/person/info/?person_id=18040"> actor_detail_url
    #         """
        actor_detail_url = actor.select_one('a').attrs['href']
        actor_detail_full_url = urljoin(actor_url,actor_detail_url )
        #print(actor_detail_url) # 상세 페이지 나옴
        print(actor_detail_full_url)


        res = requests.get(actor_detail_full_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup

        for li_tag in soup.select('ul.default_info li'): 


            ## dict의 key에 해당하는 값을 추출한다. dict['직업']
            ##<li><span class="tit">직업</span>배우</li>
            actor_item_field = li_tag.select_one('span.tit').text # w직업 생년월일 성별 등 ..

            # <span.*?>.*?</span>를 ''으로 sub 하자!!
            ## dict의 value 에 해당하는 값 추출. dict['직업']='배우'
            ##<li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) ## str로 싸서 줌 <li>배우</li>
            actor_item_value = re.sub('<.*?>','', actor_item_value) ##배우
            #print(actor_item_value)
            # url 사이의 공백 없애주기 - 멜론에서도 했었ㄷ음?

            """ 적용전
               \n http://www.gangdongwon.com
            """
            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('',actor_item_value)
            """ 적용후
               \nhttp://www.gangdongwon.com
            """

            # 이 데이터를 리스트에 저장하면 디비에 저장하기 쉽겠지?
            actor_item_dict[actor_item_field] = actor_item_value

            #print(actor_item_value)
            #print(actor_item_dict)# 이렇게 하면 누적이 됨
        actor_item_list.append(actor_item_dict)    

#len(actor_item_list) 
#actor_item_list

--->1페이지 출력됨
===>0번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=18040
===>1번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=2760
===>2번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=87306
===>3번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=167
===>4번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=71787
===>5번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=40151
===>6번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=57657
--->2페이지 출력됨
===>0번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=23841
===>1번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=25112
===>2번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=57611
===>3번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=109778
===>4번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=107320
===>5번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=1533
===>6번째 배우 출력됨
http://www.cine21.com/db/person/info/?person_id=78212
--->3페이지 출

[{'배우이름': '강동원',
  '흥행지수': 159665,
  '출연영화': ['반도'],
  '랭킹': 1,
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'배우이름': '이정현',
  '흥행지수': 139707,
  '출연영화': ['반도'],
  '랭킹': 2,
  '직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'배우이름': '이레',
  '흥행지수': 119748,
  '출연영화': ['반도'],
  '랭킹': 3,
  '직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'배우이름': '권해효',
  '흥행지수': 99795,
  '출연영화': ['반도', '사이비', '한낮의 피크닉'],
  '랭킹': 4,
  '직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'배우이름': '구교환',
  '흥행지수': 79832,
  '출연영화': ['반도'],
  '랭킹':

### 4단계
* 1년치 데이터 1905명 
* period_start가 2019-07 로해야 ..1년치..

In [82]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin
from itertools import count 
from time import sleep

actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict=dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2019-07'
formdata_dict['gender'] = 'all'


actor_item_list = list()


for page in count(1):
    formdata_dict['page'] = page
    print('--->{}페이지 출력됨'.format(page))
    #post
    res = requests.post(actor_url, data=formdata_dict)
    #print(res.ok)
    soup = BeautifulSoup(res.text, 'html.parser')

    
    actors =  soup.select('li.people_li div.name' )
    if len(actors)==0:
        break   # 빠져나갈 곳
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li span.grade')

    for index, actor in enumerate(actors): #배우랑 흥행지수
        print('===>{}번째 배우 출력됨'.format(index))
        actor_item_dict = dict()


        # 유아인(2편) 에서 (2편)을 제거하고 저장.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dict['배우이름']=actor_name
        print('--->{}({}) 페이지 출력됨'.format(actor_name, len(actor_name)))
        
        # 흥행지수 값에서 ,를 제거하고 정수타입으로 변환해서 저장, replace() ,를 공백으로
        actor_hit = int(hits[index].text.replace(',',''))
        actor_item_dict['흥행지수'] = actor_hit


        #출연작, 
        movie_titles = movies[index].select('li a span')
        # 출연작 여러개의 title을 저장하는 리스트- 권해효 3개..
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
            actor_item_dict['출연영화']= movie_title_list
        # 순위 
        actor_ranking = rankings[index].text
        actor_item_dict['랭킹']=int(actor_ranking)


    #         """
    #             <div class=name>   <= actor
    #                 <a href="/db/person/info/?person_id=18040"> actor_detail_url
    #         """
        actor_detail_url = actor.select_one('a').attrs['href']
        actor_detail_full_url = urljoin(actor_url,actor_detail_url )
        #print(actor_detail_url) # 상세 페이지 나옴
        #print(actor_detail_full_url)


        res = requests.get(actor_detail_full_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup

        for li_tag in soup.select('ul.default_info li'): 


            ## dict의 key에 해당하는 값을 추출한다. dict['직업']
            ##<li><span class="tit">직업</span>배우</li>
            actor_item_field = li_tag.select_one('span.tit').text # w직업 생년월일 성별 등 ..

            # <span.*?>.*?</span>를 ''으로 sub 하자!!
            ## dict의 value 에 해당하는 값 추출. dict['직업']='배우'
            ##<li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(li_tag)) ## str로 싸서 줌 <li>배우</li>
            actor_item_value = re.sub('<.*?>','', actor_item_value) ##배우
            #print(actor_item_value)
            # url 사이의 공백 없애주기 - 멜론에서도 했었ㄷ음?

            """ 적용전
               \n http://www.gangdongwon.com
            """
            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('',actor_item_value)
            """ 적용후
               \nhttp://www.gangdongwon.com
            """

            # 이 데이터를 리스트에 저장하면 디비에 저장하기 쉽겠지?
            actor_item_dict[actor_item_field] = actor_item_value

            #print(actor_item_value)
            #print(actor_item_dict)# 이렇게 하면 누적이 됨
        actor_item_list.append(actor_item_dict)    

#len(actor_item_list) 
#actor_item_list

--->1페이지 출력됨
===>0번째 배우 출력됨
--->이병헌(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=947
===>1번째 배우 출력됨
--->하정우(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=21119
===>2번째 배우 출력됨
--->마동석(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=64614
===>3번째 배우 출력됨
--->전혜진(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=11198
===>4번째 배우 출력됨
--->이성민(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=25123
===>5번째 배우 출력됨
--->김아중(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=24464
===>6번째 배우 출력됨
--->박정민(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=74826
--->2페이지 출력됨
===>0번째 배우 출력됨
--->강기영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99508
===>1번째 배우 출력됨
--->조진웅(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=60358
===>2번째 배우 출력됨
--->조정석(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=71348
===>3번째 배우 출력됨
--->배수지(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=74503
===>4번째 

===>1번째 배우 출력됨
--->송강호(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=2263
===>2번째 배우 출력됨
--->한석규(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=813
===>3번째 배우 출력됨
--->안성기(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=346
===>4번째 배우 출력됨
--->김고은(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=79219
===>5번째 배우 출력됨
--->차승원(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=7006
===>6번째 배우 출력됨
--->김혜준(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=104624
--->15페이지 출력됨
===>0번째 배우 출력됨
--->라미란(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=48845
===>1번째 배우 출력됨
--->박영규(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=10321
===>2번째 배우 출력됨
--->신구(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=1068
===>3번째 배우 출력됨
--->이채은(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=69097
===>4번째 배우 출력됨
--->허율(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=104967
===>5번째 배우 출력됨
--->김홍파(3)

===>2번째 배우 출력됨
--->서현우(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=88173
===>3번째 배우 출력됨
--->이준혁(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=74920
===>4번째 배우 출력됨
--->RM(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=101169
===>5번째 배우 출력됨
--->안소이(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=79649
===>6번째 배우 출력됨
--->이현(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=94998
--->28페이지 출력됨
===>0번째 배우 출력됨
--->박성훈(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=104560
===>1번째 배우 출력됨
--->윤여정(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=13093
===>2번째 배우 출력됨
--->이호정(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=101931
===>3번째 배우 출력됨
--->이원근(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=87512
===>4번째 배우 출력됨
--->배정남(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=70796
===>5번째 배우 출력됨
--->최광일(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=17215
===>6번째 배우 출력됨
--

===>2번째 배우 출력됨
--->정인기(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=40163
===>3번째 배우 출력됨
--->임채헌(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=85816
===>4번째 배우 출력됨
--->정상훈(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=27697
===>5번째 배우 출력됨
--->김새벽(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=83063
===>6번째 배우 출력됨
--->윤박(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=86951
--->41페이지 출력됨
===>0번째 배우 출력됨
--->이시언(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=70680
===>1번째 배우 출력됨
--->이정은(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=98342
===>2번째 배우 출력됨
--->이찬유(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=109591
===>3번째 배우 출력됨
--->장혁진(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=40158
===>4번째 배우 출력됨
--->정승욱(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108088
===>5번째 배우 출력됨
--->김혜성(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=95001
===>6번째 배우 출력됨
--

===>2번째 배우 출력됨
--->이창민(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108383
===>3번째 배우 출력됨
--->정국(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=100827
===>4번째 배우 출력됨
--->김나연(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=109303
===>5번째 배우 출력됨
--->장병관(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=100999
===>6번째 배우 출력됨
--->황창영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=100290
--->54페이지 출력됨
===>0번째 배우 출력됨
--->이소영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=88419
===>1번째 배우 출력됨
--->심은경(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=62512
===>2번째 배우 출력됨
--->박지영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=50825
===>3번째 배우 출력됨
--->장원영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=73847
===>4번째 배우 출력됨
--->강성우(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=109818
===>5번째 배우 출력됨
--->한으뜸(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=110208
===>6번째 배우 출

...

===>5번째 배우 출력됨
--->최선자(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=17122
===>6번째 배우 출력됨
--->서정(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=1402
--->209페이지 출력됨
===>0번째 배우 출력됨
--->이은주(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=15240
===>1번째 배우 출력됨
--->신정훈(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=96630
===>2번째 배우 출력됨
--->강능원(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108777
===>3번째 배우 출력됨
--->김동희(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=71834
===>4번째 배우 출력됨
--->오의식(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99658
===>5번째 배우 출력됨
--->김동건(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=70369
===>6번째 배우 출력됨
--->곽두조(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=107529
--->210페이지 출력됨
===>0번째 배우 출력됨
--->오의식(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99658
===>1번째 배우 출력됨
--->김동건(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=70369
==

===>5번째 배우 출력됨
--->황윤(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=59000
===>6번째 배우 출력됨
--->이보영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=103664
--->222페이지 출력됨
===>0번째 배우 출력됨
--->천보근(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=70823
===>1번째 배우 출력됨
--->허기호(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=423
===>2번째 배우 출력됨
--->홍윤정(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=77625
===>3번째 배우 출력됨
--->최휘찬(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=98339
===>4번째 배우 출력됨
--->박세진(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108169
===>5번째 배우 출력됨
--->나영희(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=12208
===>6번째 배우 출력됨
--->박은숙(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=8830
--->223페이지 출력됨
===>0번째 배우 출력됨
--->서강준(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=87470
===>1번째 배우 출력됨
--->이일웅(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=419
===>2번

===>5번째 배우 출력됨
--->하덕성(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=30106
===>6번째 배우 출력됨
--->김현빈(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=103391
--->235페이지 출력됨
===>0번째 배우 출력됨
--->정현(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=102046
===>1번째 배우 출력됨
--->최윤빈(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=111981
===>2번째 배우 출력됨
--->최진영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108434
===>3번째 배우 출력됨
--->반효정(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=47796
===>4번째 배우 출력됨
--->정진수(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=409
===>5번째 배우 출력됨
--->이수현(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=93095
===>6번째 배우 출력됨
--->조하영(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=93954
--->236페이지 출력됨
===>0번째 배우 출력됨
--->강하나(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99107
===>1번째 배우 출력됨
--->김지훈(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=25427
=

===>5번째 배우 출력됨
--->한하나(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99995
===>6번째 배우 출력됨
--->박지수(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=82079
--->248페이지 출력됨
===>0번째 배우 출력됨
--->박혜숙(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=17850
===>1번째 배우 출력됨
--->임양(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=49223
===>2번째 배우 출력됨
--->진상진(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108591
===>3번째 배우 출력됨
--->이은성(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=47035
===>4번째 배우 출력됨
--->장현성(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=15147
===>5번째 배우 출력됨
--->한하나(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99995
===>6번째 배우 출력됨
--->박지수(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=82079
--->249페이지 출력됨
===>0번째 배우 출력됨
--->여호민(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=59370
===>1번째 배우 출력됨
--->한성수(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=100990
=

===>6번째 배우 출력됨
--->이영미(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=62955
--->261페이지 출력됨
===>0번째 배우 출력됨
--->김민수(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=99519
===>1번째 배우 출력됨
--->강문(2) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=46903
===>2번째 배우 출력됨
--->박용팔(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=701
===>3번째 배우 출력됨
--->김도현(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=90110
===>4번째 배우 출력됨
--->윤은지(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=101256
===>5번째 배우 출력됨
--->남상지(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=103587
===>6번째 배우 출력됨
--->김종원(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=108150
--->262페이지 출력됨
===>0번째 배우 출력됨
--->이세미(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=110217
===>1번째 배우 출력됨
--->이상호(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=93995
===>2번째 배우 출력됨
--->김세정(3) 페이지 출력됨
http://www.cine21.com/db/person/info/?person_id=111455


In [87]:
print(len(actor_item_list))
actor_final_list = [actor for actor in actor_item_list if len(actor['배우이름'])< 10]
#print(actor_final_list) #중간에 배우 이름말고 이상한거 껴있을때 배우만 뽑아내려고..

1905
